In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Hack').getOrCreate()
dataset = spark.read.csv('dbfs:/FileStore/shared_uploads/gkantirisrafael@gmail.com/hack_data.csv',header=True,inferSchema=True)
dataset.head(1)

Out[2]: [Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

In [0]:
dataset.columns

Out[4]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [0]:
feat_col = ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [0]:
assembler = VectorAssembler(inputCols=feat_col,outputCol='features')

In [0]:
final_data = assembler.transform(dataset)

In [0]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features',outputCol='ScaledFeatures')

In [0]:
scaler_model = scaler.fit(final_data)
cluster_final_data = scaler_model.transform(final_data)

In [0]:
cluster_final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- ScaledFeatures: vector (nullable = true)



In [0]:
kmeans2 = KMeans(featuresCol='ScaledFeatures',k=2)
kmeans3 = KMeans(featuresCol='ScaledFeatures',k=3)

In [0]:
model_k3 = kmeans3.fit(cluster_final_data)
model_k2 = kmeans2.fit(cluster_final_data)

In [0]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|   84|
|         0|  167|
+----------+-----+



In [0]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

